**6. Preprocess Data and Create Labels and Features**

In [1]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 30.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join ('.MP_Data')

# Actions that we try to detect
actions = np.array(['again','boy','deaf','finish','girl','go','goodbye','hearing','hello','help',
'how','i','know','like','love','man','more','my','need','no',
'none','not','please','right','school','science','sentence','sign-language','sorry','sports',
'student','thank-you','that','want','what','when','where','who','why','with',
'without','woman','yes','you','your','i-love-you','youre-welcome','intelligent','friend','come-here',
'go-away','see-you-later','baby','name'])

no_sequences = 60 # No. of sequences per gesture
sequence_length = 60 # No. of frames in each video sequence

In [4]:
label_map = {label: num for num, label in enumerate (actions)} # Mapping labels to numbers

In [5]:
label_map # Mapping labels to numbers because we can't train on strings

{'again': 0,
 'boy': 1,
 'deaf': 2,
 'finish': 3,
 'girl': 4,
 'go': 5,
 'goodbye': 6,
 'hearing': 7,
 'hello': 8,
 'help': 9,
 'how': 10,
 'i': 11,
 'know': 12,
 'like': 13,
 'love': 14,
 'man': 15,
 'more': 16,
 'my': 17,
 'need': 18,
 'no': 19,
 'none': 20,
 'not': 21,
 'please': 22,
 'right': 23,
 'school': 24,
 'science': 25,
 'sentence': 26,
 'sign-language': 27,
 'sorry': 28,
 'sports': 29,
 'student': 30,
 'thank-you': 31,
 'that': 32,
 'want': 33,
 'what': 34,
 'when': 35,
 'where': 36,
 'who': 37,
 'why': 38,
 'with': 39,
 'without': 40,
 'woman': 41,
 'yes': 42,
 'you': 43,
 'your': 44,
 'i-love-you': 45,
 'youre-welcome': 46,
 'intelligent': 47,
 'friend': 48,
 'mother': 49,
 'father': 50,
 'see-you-later': 51,
 'baby': 52,
 'name': 53}

In [6]:
sequences, labels = [], [] # Empty lists to store the sequences and labels
for action in actions: # Looping through the gestures
    for sequence in range(no_sequences): # Iterate over the 30 videos per gesture
        window = [] # Empty window to store the features
        for frame_num in range(sequence_length): # Iterate over the 30 frames per video
            res = np.load(os.path.join (DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))) # Loading the numpy array
            window.append(res) # Appending the features of the frame to the window
        sequences.append(window) # Appending the window to the sequences list
        labels.append(label_map[action]) # Appending the label to the labels list

In [7]:
len(sequences) # Total number of samples

1620

In [8]:
np.array(sequences).shape # Shape of the numpy array containing all the sequences

(1620, 30, 1662)

In [9]:
len(labels) # Total number of labels

1620

In [10]:
np.array(labels).shape # Shape of the numpy array containing all the labels

(1620,)

In [11]:
X = np.array(sequences) # Converting the sequences list to numpy array

In [12]:
X  

array([[[ 5.06265163e-01,  3.88014466e-01, -6.59309089e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.06795108e-01,  3.86388242e-01, -5.78921318e-01, ...,
          3.57786179e-01,  6.63773835e-01, -3.68799530e-02],
        [ 5.06824911e-01,  3.86062592e-01, -5.66417217e-01, ...,
          3.59249026e-01,  6.65324509e-01, -4.02773395e-02],
        ...,
        [ 5.10434747e-01,  3.95760208e-01, -4.94679600e-01, ...,
          4.96528208e-01,  7.43361831e-01, -1.10606337e-02],
        [ 5.10493457e-01,  3.95746499e-01, -4.86615419e-01, ...,
          4.96858090e-01,  7.44323730e-01, -1.00633241e-02],
        [ 5.10489821e-01,  3.95287931e-01, -4.86658394e-01, ...,
          4.96576577e-01,  7.43771315e-01, -1.33194104e-02]],

       [[ 5.10641932e-01,  3.94899338e-01, -4.91338491e-01, ...,
          4.95887995e-01,  7.45105147e-01, -8.27397127e-03],
        [ 5.10233879e-01,  3.94847989e-01, -4.64167029e-01, ...,
          3.20699304e-01,  5.25600433e

In [13]:
X.shape 

(1620, 30, 1662)

In [14]:
y = to_categorical(labels).astype(int) # One-hot encoding the labels because we are using categorical_crossentropy loss

In [15]:
y # One-hot encoded labels because we can't train on strings

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [16]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.05) # Splitting the data into training and testing sets

**7. Build and Train LSTM Neural Network**

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

2022-09-29 04:50:31.373608: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-09-29 04:50:31.374414: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-09-29 04:50:31.444204: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-09-29 04:50:31.486041: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-09-29 04:50:31.486195: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


In [19]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ./ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2EiYBg2SbWRDXybCT6huKzEzYH6_3zWcijJrMgWFSBwUpkjt2

--2022-09-29 04:50:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  8.26MB/s    in 1.6s    

2022-09-29 04:50:34 (8.26 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ./ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:
import os
import multiprocessing
 
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./Logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/proces

In [21]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9f2a-34-74-25-61.ngrok.io


In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2022-09-29 04:50:38.354013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 04:50:38.463810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 04:50:38.464981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-29 04:50:38.467384: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [23]:
[actions.shape[0]]

[54]

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

TensorBoard 2.10.0 at http://0.0.0.0:6006/ (Press CTRL+C to quit)
2022-09-29 04:50:43.329885: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40000
 1/49 [..............................] - ETA: 2:58 - loss: 3.9770 - categorical_accuracy: 0.0312

2022-09-29 04:50:47.182417: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-09-29 04:50:47.182462: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/49 [>.............................] - ETA: 27s - loss: 4.4206 - categorical_accuracy: 0.0156 

2022-09-29 04:50:47.693141: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-09-29 04:50:47.698961: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-09-29 04:50:47.878376: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 6299 callback api events and 6296 activity events. 
2022-09-29 04:50:48.029891: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-09-29 04:50:48.265345: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: Logs/train/plugins/profile/2022_09_29_04_50_48

2022-09-29 04:50:48.449611: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.trace.json.gz


 3/49 [>.............................] - ETA: 39s - loss: 5.1749 - categorical_accuracy: 0.0104

2022-09-29 04:50:48.618094: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: Logs/train/plugins/profile/2022_09_29_04_50_48

2022-09-29 04:50:48.631536: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.memory_profile.json.gz
2022-09-29 04:50:48.635200: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: Logs/train/plugins/profile/2022_09_29_04_50_48
Dumped tool data for xplane.pb to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.xplane.pb
Dumped tool data for overview_page.pb to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.overview_page.pb
Dumped tool data for input_pipeline.pb to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.input_pipeline.pb
Dumped tool data for tensorflow_stats.pb to Logs/train/plugins/profile/2022_09_29_04_50_48/de581a55cf04.tensorflow_stats.

49/49 [==============================] - 13s 191ms/step - loss: 4.7441 - categorical_accuracy: 0.0175
Epoch 2/40000
49/49 [==============================] - 8s 172ms/step - loss: 3.9916 - categorical_accuracy: 0.0214
Epoch 3/40000
49/49 [==============================] - 9s 192ms/step - loss: 3.9899 - categorical_accuracy: 0.0195
Epoch 4/40000
49/49 [==============================] - 8s 160ms/step - loss: 3.9869 - categorical_accuracy: 0.0201
Epoch 5/40000
49/49 [==============================] - 8s 172ms/step - loss: 3.9839 - categorical_accuracy: 0.0188
Epoch 6/40000
49/49 [==============================] - 8s 162ms/step - loss: 3.9774 - categorical_accuracy: 0.0253
Epoch 7/40000
49/49 [==============================] - 7s 150ms/step - loss: 3.8886 - categorical_accuracy: 0.0474
Epoch 8/40000
49/49 [==============================] - 8s 163ms/step - loss: 3.5460 - categorical_accuracy: 0.0390
Epoch 9/40000
49/49 [==============================] - 8s 172ms/step - loss: 3.2694 - categor

KeyboardInterrupt: 

Process ForkPoolWorker-19:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
Process ForkPoolWorker-17:
KeyboardInterrupt
Process ForkPoolWorker-11:
Process ForkPoolWorker-20:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-12:
Traceback (most recent call last):
Process Fo

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 54)                1782      
Total params: 598,358
Trainable params: 598,358
Non-trainable params: 0
__________________________________________________

**9. Make Predictions**

In [27]:
res = model.predict(X_train)

In [28]:
np.sum(res[0])

1.0

In [64]:
actions[np.argmax(res[4])]

'need'

In [65]:
actions[np.argmax(y_train[4])]

'deaf'

In [67]:
model.save('UpdatedAction.h5')

**10. Evaluation using Confusion Matrix and Accuracy**

In [69]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [77]:
yhat = model.predict(X_train)

In [78]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [79]:
ytrue

[42,
 21,
 31,
 3,
 43,
 52,
 39,
 39,
 37,
 45,
 46,
 16,
 1,
 20,
 34,
 45,
 21,
 44,
 6,
 35,
 26,
 23,
 43,
 22,
 42,
 33,
 31,
 2,
 20,
 12,
 36,
 0,
 53,
 6,
 12,
 0,
 2,
 9,
 22,
 27,
 25,
 26,
 32,
 41,
 29,
 6,
 43,
 31,
 25,
 18,
 35,
 24,
 4,
 6,
 45,
 7,
 3,
 25,
 45,
 4,
 8,
 1,
 44,
 12,
 29,
 33,
 50,
 23,
 25,
 13,
 51,
 8,
 51,
 7,
 3,
 23,
 7,
 5,
 32,
 15,
 28,
 4,
 34,
 47,
 51,
 53,
 41,
 16,
 50,
 33,
 21,
 52,
 0,
 22,
 28,
 30,
 36,
 52,
 7,
 17,
 32,
 46,
 46,
 9,
 7,
 22,
 5,
 45,
 18,
 26,
 36,
 24,
 14,
 0,
 0,
 30,
 32,
 23,
 6,
 53,
 13,
 50,
 7,
 28,
 22,
 23,
 27,
 2,
 50,
 1,
 40,
 0,
 2,
 18,
 2,
 44,
 46,
 36,
 30,
 46,
 11,
 36,
 21,
 18,
 26,
 32,
 21,
 49,
 6,
 26,
 0,
 1,
 33,
 29,
 10,
 22,
 35,
 35,
 13,
 16,
 12,
 3,
 1,
 16,
 23,
 45,
 38,
 23,
 53,
 44,
 3,
 21,
 11,
 5,
 15,
 8,
 33,
 24,
 51,
 48,
 42,
 51,
 29,
 35,
 12,
 10,
 17,
 51,
 5,
 26,
 38,
 37,
 5,
 30,
 27,
 24,
 25,
 18,
 34,
 17,
 9,
 21,
 9,
 49,
 42,
 43,
 28,
 38,
 4,
 15,


In [80]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1510,    0],
        [   0,   29]],

       [[1508,    1],
        [  11,   19]],

       [[1508,    4],
        [   8,   19]],

       [[1510,    0],
        [   0,   29]],

       [[1512,    0],
        [  11,   16]],

       [[1509,    0],
        [   0,   30]],

       [[1500,   11],
        [   1,   27]],

       [[1509,    1],
        [   2,   27]],

       [[1507,    3],
        [   0,   29]],

       [[1510,    0],
        [  19,   10]],

       [[1509,    0],
        [   0,   30]],

       [[1507,    4],
        [   7,   21]],

       [[1510,    0],
        [   9,   20]],

       [[1506,    6],
        [   4,   23]],

       [[1509,    0],
        [   5,   25]],

       [[1511,    0],
        [   1,   27]],

       [[1506,    4],
        [   0,   29]],

       [[1506,    4],
        [   3,   26]],

       [[1499,   13],
        [   1,   26]],

       [[1512,    0],
        [   1,   26]],

       [[1511,    0],
        [   0,   28]],

       [[1510,    0],
        [   

In [81]:
accuracy_score(ytrue, yhat)

0.9259259259259259